# Getting the most out of NSIDC DAAC data: Harmonizing disparate data for land and sea ice applications

This tutorial will walk you through data discovery, access, and analysis of NSIDC DAAC data...


Here are the steps you will learn in this tutorial:
  
   * Learn the basics of Jupyter Lab and Jupyter Notebook
   * Visualize time and area of interest using NASA Worldview / OpenAltimetry
   * Search for data programmatically by time and area of interest.
   * Determine subsetting and reformatting capabilities for our data of interest.
   * Access and customize data using NSIDC's API service.
       * ICESat-2
       * CETB
       * Ice velocity
       * Passive microwave
   * Plot ATL03 photon data along with higher level ATL06/ATL10 data
   * Filter ICESat-2 data by quality flag
   * Use xarray to extract gridded data along ICESat-2 tracks.
   * Plot data with MODIS true color WMS endpoint. 



### Import Packages

In [1]:
import requests
import getpass
import socket
import json
import zipfile
import io
import math
import os
import shutil
import pprint
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import fiona
import h5py
import re
import import_ipynb
from ipynb.fs.full.Functions import *
# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
from statistics import mean
from requests.auth import HTTPBasicAuth

### Introduction to NSIDC and NSIDC data resources



### Search for data programmatically by time and area of interest.


In [2]:
# Input data set short name (example: ATL06)
dataset = 'ATL06'

In [3]:
# Find all instances of 'version_id' in metadata and print most recent version number
params = {
    'short_name': dataset
}
cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)
# Find most recent version number
versions = [el['version_id'] for el in results['feed']['entry']]
latest_version = max(versions)
print('The most recent version of ', dataset, ' is ', latest_version)

The most recent version of  ATL06  is  001


In [4]:
# Input temporal range 

# Input start date in yyyy-MM-dd format
start_date = '2019-02-22'
# Input start time in HH:mm:ss format
start_time = '00:00:00'
# Input end date in yyyy-MM-dd format
end_date = '2019-05-22'
# Input end time in HH:mm:ss format
end_time = '23:59:59'

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'
print(temporal)


2019-02-22T00:00:00Z,2019-05-22T23:59:59Z


In [5]:
# Bounding Box spatial parameter in 'W,S,E,N' format

# Input lower left longitude in decimal degrees
LL_lon = '-64'
# Input lower left latitude in decimal degrees
LL_lat = '66'
# Input upper right longitude in decimal degrees
UR_lon = '-55'
# Input upper right latitude in decimal degrees
UR_lat = '72'

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat
print(bounding_box)

-64,66,-55,72


In [6]:
data_search(dataset, latest_version, temporal, bounding_box)

There are 67 granules of ATL06 version 001 over my area and time of interest.
The average size of each granule is 34.22 MB and the total size of all 67 granules is 2292.81 MB


Note that subsetting, reformatting, or reprojecting can alter the size of the granules if those services are applied to your request.

### Input Earthdata Login credentials

An Earthdata Login account is required to access data from the NSIDC DAAC. If you do not already have an Earthdata Login account, visit http://urs.earthdata.nasa.gov to register.

In [7]:
# Earthdata Login credentials

uid = input('Earthdata Login user name: ')
pswd = getpass.getpass('Earthdata Login password: ')
email = input('Email address associated with Earthdata Login account: ')

Earthdata Login user name:  amy.steiker
Earthdata Login password:  ·········
Email address associated with Earthdata Login account:  amy.steiker@nsidc.org


### Select the subsetting, reformatting, and reprojection services enabled for your data set of interest.

The NSIDC DAAC supports customization services on many of our NASA Earthdata mission collections. Let's discover whether or not our data set has these services available. If services are available, we will also determine the specific service options supported for this data set and select which of these services we want to request.

In [8]:
service_search(dataset, latest_version, bounding_box, temporal, uid, pswd)

Subsetting by bounding box, based on the area of interest inputted above, is available. Would you like to request this service? (y/n) y
Subsetting by time, based on the temporal range inputted above, is available. Would you like to request this service? (y/n) y


These reformatting options are available: ['TABULAR_ASCII', 'NetCDF4-CF', 'Shapefile', 'NetCDF-3']


If you would like to reformat, copy and paste the reformatting option you would like (make sure to omit quotes, e.g. GeoTIFF), otherwise leave blank. 


No reprojection options are supported with your requested format


### Access and customize data using NSIDC's API



- Feed data into API request
- Print API requests 
- Put data in Outputs folder

### Sea Ice use case: Explore how ICESat-2 can help map fast ice

Cycle 2, Track 1298, March 23, 2019. The track crosses the East Siberian Sea, going from land to fast ice, to drift ice including some new ice growth, and is nearly completely clear sky. The thought here is that ICESat-2 could potentially help map fast ice, which is rather difficult to do automatically with satellite imagery. My thought is that the fast ice should have less height/freeboard variability - less surface roughness. It also crosses new ice in a flaw lead as the drift ice moves away from the fast ice edge. So there is a good graduation of ice thickness from new ice to thicker first-year ice. So this could show how well ICESat-2 performs at different ice thicknesses. We could use MODIS reflectance to estimate thin ice thickness, and/or we could also use SMAP. The other neat thing is that there is a cross-over with another track, 1305, on the same day. So could look at the cross-over and compare elevation/freeboard. This point is over drift ice, so it likely wouldn’t be the same floe since it’s 7 orbits later (~12 hours), but if nothing else it would demonstrate how to compare cross-over. (We could use all six beams to get the full 36 crossovers, or maybe just the 3 strong beams to get 9 crossovers - that might be able to capture some of the same floes).

 
1. CMR search to find MYD29E1D.6, SMAP (what data set?), and MODIS IST (MOD29) over East Siberian Sea bounding box over 3/23/19
2. Look on OA and/or Worldview to confirm coverage
3. Determine projections, formats, file structures, and parameters
4. Set customization options
5. Access data
6. Linear relationship: Apply empirical formulas using coefficients for water and ice
7. Comparing cross overs
8. Plot results

### Land Ice use case: Correlate velocity with elevation?

Cycle 1, Track 285, Oct. 17, 2018
Cycle 3, Track 338, Apr. 20, 2019 - might have some clouds
Cycle 2, Track 1283, Mar. 22, 2019
Cycle 2, Track 1291, Mar. 23, 2019 - this crosses the mouth of Jakobshavn, not sure if it would catch the edge of the ice tongue, but may get some sea ice as well



### Data filtering by quality flag




### Xarray to extract gridded data along ICESat-2 tracks


### Plot data with MODIS true color WMS endpoint

In [9]:
url = 'https://gibs.earthdata.nasa.gov/twms/epsg4326/best/twms.cgi?request=GetMap&layers=MODIS_Terra_CorrectedReflectance_TrueColor&srs=EPSG:4326&format=image/jpeg&styles=&time=2012-07-09&width=512&height=512&bbox=-18,27,-13.5,31.5'